<a href="https://colab.research.google.com/github/Yohanes213/Hamoye/blob/main/Hamoye_2_Machine_Learning_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv('energydata_complete.csv')

In [4]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [43]:
X = df["T2"].to_frame()
y = df["T6"]

In [45]:
linear_model = LinearRegression()

In [46]:
linear_model.fit(X,y)

LinearRegression()

In [47]:
y_pred1 = linear_model.predict(X)

In [50]:
from sklearn.metrics import r2_score
r2_score1 = r2_score(y, y_pred1)
round(r2_score1, 2)
     

0.64

In [113]:
from sklearn.preprocessing import MinMaxScaler

In [114]:
scaler = MinMaxScaler()
new_df = df.drop(columns=['date', 'lights'])
normalised_df = pd.DataFrame(scaler.fit_transform(new_df), columns=new_df.columns)
X = normalised_df.drop(columns=['Appliances'])
y = normalised_df['Appliances']

In [115]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
     

In [116]:
linear_model2 = LinearRegression()
linear_model2.fit(x_train,y_train)

LinearRegression()

In [117]:
y_pred = linear_model2.predict(x_test)

In [118]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
round(mae, 2)

0.05

In [119]:
rss = np.sum(np.square(y_test - y_pred))
round(rss, 2) 
     

45.35

In [120]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
round(mse, 3)

0.008

In [121]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, y_pred)
round(r2_score, 2)

0.15

In [122]:
def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weight_df = pd.DataFrame(weights).reset_index()
  weight_df.columns = ['Features', col_name]
  weight_df[col_name].round(3)
  return weight_df

In [123]:
linear_model_weights = get_weights_df(linear_model2, x_train,'Linear Model Weights')

In [139]:
linear_model_weights[linear_model_weights['Linear Model Weights'] == linear_model_weights['Linear Model Weights'].max()]

,Features,Linear Model Weights
25,RH_1,0.553547


In [141]:
linear_model_weights[linear_model_weights['Linear Model Weights'] == linear_model_weights['Linear Model Weights'].min()]

,Features,Linear Model Weights
0,RH_2,-0.456698


In [125]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=0.4)
ridge.fit(x_train, y_train)

Ridge(alpha=0.4)

In [126]:
y_ridge = ridge.predict(x_test)

In [127]:
mse = mean_squared_error(y_test, y_ridge)
round(mse, 3)

0.008

In [128]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.001)
lasso.fit(x_train, y_train)

Lasso(alpha=0.001)

In [129]:
lasso_model_weights = get_weights_df(lasso, x_train,'Lasso Model Weights')

In [130]:
len(lasso_model_weights[lasso_model_weights['Lasso Model Weights'] != 0])

4

In [132]:
y_lasso = lasso.predict(x_test)

In [133]:
rmse = np.sqrt(mean_squared_error(y_test, y_lasso))
round(rmse, 3) 

0.094